In [1]:
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from itertools import combinations
from collections import Counter

In [2]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [3]:
okt = Okt()

In [4]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [5]:
def extract_keywords(text, num_keywords=10):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    return [keyword for keyword, _ in top_keywords]

In [23]:
def parse_text_file(file_path):
    result_dict = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄의 앞뒤 공백 제거
            line = line.strip()

            # C?? 형식의 코드와 나머지 문자열 분리
            if line:
                parts = line.split(' ', 1)  # 첫 번째 공백으로 나눔
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(',')  # 쉼표와 공백으로 구분하여 리스트 생성
                    result_dict[key] = values

    return result_dict

In [24]:
def count_matches(keyword_list, reference_list):
    return sum(1 for keyword in keyword_list if keyword in reference_list)

In [6]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [7]:
len(df)

3611424

In [8]:
df.columns

Index(['id', 'topic', 'text'], dtype='object')

In [9]:
df_t = df[df.topic == '기술']

In [10]:
del df

In [11]:
len(df_t)

382513

In [12]:
documents = df_t.text

In [13]:
doc_keywords = []

for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

100%|████████████████████████████████| 382513/382513 [21:52:37<00:00,  4.86it/s]


In [14]:
doc_keywords[:100]

[['지수', '사회', '공존', '사람', '지능지수', '능력', '위해', '다른', '인맥', '모두'],
 ['우주', '센터', '나로', '고흥', '나로도', '위성', '발사', '발사체', '소록도', '해변'],
 ['인간', '아집', '독선', '파스칼', '사회', '자신', '때문', '우주', '자연', '사람'],
 ['시민', '천문대', '천문학', '대해', '사람', '최초', '교육', '과장', '우주', '아주'],
 ['주제', '감촉', '강의', '촉감', '오늘', '화석연료', '소비', '한자', '때문', '토씨'],
 ['소니', '업적', '기록', '일본', '전자', '시장', '적자', '영업', '부문', '미국'],
 ['노트북', '사용', '액정', '고장', '부품', '삼보', '컴퓨터', '에버라텍', '구입', '중고'],
 ['합병', '시장', '사장', '경쟁', '통신', '독점', '텔레콤', '유선', '기업', '정만원'],
 ['회선', '인터넷', '정부', '위약금', '사용', '위해', '지원', '유치', '회원', '정책'],
 ['밥물', '가장', '밥솥', '냄새', '예전', '이상', '세상', '식사', '준비', '시절'],
 ['적벽대전', '조조', '역사', '제갈량', '주유', '전쟁', '유비', '인물', '삼국지', '손권'],
 ['부품', '교체', '중고', '수리', '액정', '휴대전화', '교환', '기사', '사용', '생각'],
 ['아이', '평화', '꼬리별', '전하', '이야기', '사람', '메시지', '이제', '피리', '카스'],
 ['휴대전화', '복제', '휴대폰', '사생활', '등록', '전파', '연예인', '개인정보', '자신', '불법'],
 ['난청', '소음', '청력', '정도', '노출', '경우', '음성', '손상', '대화', '이상'],
 ['어머니', '우리', '얼굴', 

In [16]:
len(df_t)

382513

In [17]:
len(doc_keywordswords)

382513

In [18]:
df_t['keywords'] = doc_keywords

In [20]:
df_t[:10]

,id,topic,text,keywords
9,NIRW1900000001.17,기술,"모두에게 '윈-윈 게임'이 되는 세상, 아름다운 동행 21세기는 IQ(지능지수, I...","[지수, 사회, 공존, 사람, 지능지수, 능력, 위해, 다른, 인맥, 모두]"
106,NIRW1900000001.215,기술,자동차 타고 우주로 드라이브할까 날씨가 추워지면서 일상생활이 많이 움츠러든다. 바깥...,"[우주, 센터, 나로, 고흥, 나로도, 위성, 발사, 발사체, 소록도, 해변]"
249,NIRW1900000001.432,기술,"독선과 아집은 왜 생기는 걸까? <팡세>에 관한 단상 1. ""인간은 자연에서, 그것...","[인간, 아집, 독선, 파스칼, 사회, 자신, 때문, 우주, 자연, 사람]"
317,NIRW1900000001.552,기술,"""학업에 치친 청소년들도 멋진 별세계에 찾아오세요"" 국내최초로 일반인들을 대상으로 ...","[시민, 천문대, 천문학, 대해, 사람, 최초, 교육, 과장, 우주, 아주]"
354,NIRW1900000001.620,기술,'-의' 안 써야 우리 말이 깨끗하다 (247) ㄱ. 소비의 엄청난 증가로 인해 ....,"[주제, 감촉, 강의, 촉감, 오늘, 화석연료, 소비, 한자, 때문, 토씨]"
371,NIRW1900000001.655,기술,"일본 전자의 '간판' 소니, 14년 만에 영업 적자 일본 전자산업의 '얼굴'이라 할...","[소니, 업적, 기록, 일본, 전자, 시장, 적자, 영업, 부문, 미국]"
393,NIRW1900000001.686,기술,중고 노트북도 이젠 합체하게 해 주세요! 하루가 다르게 기술 개발이 이루어지고 신제...,"[노트북, 사용, 액정, 고장, 부품, 삼보, 컴퓨터, 에버라텍, 구입, 중고]"
599,NIRW1900000001.1032,기술,"이석채 ""KT-KTF 합병, 뭐가 문제냐?""정만원 ""지금 독점기업 만들 때인가?"" ...","[합병, 시장, 사장, 경쟁, 통신, 독점, 텔레콤, 유선, 기업, 정만원]"
623,NIRW1900000001.1079,기술,"""정보통신부가 지원합니다...""정부 팔아 거짓영업, 이게 뭡니까 며칠 전 인터넷통신...","[회선, 인터넷, 정부, 위약금, 사용, 위해, 지원, 유치, 회원, 정책]"
719,NIRW1900000001.1278,기술,밥물 끓는 냄새 전기압력밥솥이 급경사를 오르는 증기기관차처럼 헉헉거리며 스팀을 뿜어...,"[밥물, 가장, 밥솥, 냄새, 예전, 이상, 세상, 식사, 준비, 시절]"


In [22]:
df_t.to_csv(directory_path+'/combined_news_T.tsv', sep='\t', index=False)

In [25]:
file_path = '/home/osung/Downloads/KSIC_keyword.txt'
KSIC_key = parse_text_file(file_path)

In [26]:
combined_list = []

for key in KSIC_key :
    combined_list.extend(KSIC_key[key])

In [29]:
# 각 행에 대해 일치하는 문자열의 개수를 세고 새로운 열에 저장
df_t['match_count'] = df_t['keywords'].apply(lambda x: count_matches(x, combined_list))

In [30]:
df = df_t.sort_values(by='match_count', ascending=False)

In [31]:
df[:100]

,id,topic,text,keywords,match_count
3329384,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"[부품, 전자, 애플, 아이폰, 스마트폰, 삼성, 갤럭시, 반도체, 더욱, 메모리]",8
1746123,NPRW1900000055.24070,기술,"""스마트폰이 IT 중심…무조건 올라타야"" 삼성전기 사장의 로켓론 ◆ 갤럭시 쏠림의 ...","[삼성, 전자, 갤럭시, 스마트폰, 로켓, 부품, 모바일, 디스플레이, 휴대폰, 계열]",8
534453,NPRW1900000067.3530,기술,"삼성전자, 프리미엄 디지털 엑스레이 ‘GC85A’ 출시 삼성전자가 사용성과 영상품질...","[전자, 엑스레이, 디지털, 의료, 촬영, 환자, 영상, 진단, 프리미엄, 품질]",7
136187,NIRW1900000029.1965,기술,갤럭시S8 vs 아이폰7…당신의 선택은? 삼성전자의 플래그십 스마트폰 갤럭시S8이 ...,"[갤럭시, 아이폰, 성능, 탑재, 배터리, 디스플레이, 카메라, 스마트폰, 애플, 적용]",7
1778105,NPRW1900000057.139,기술,공중 화장실용 항균비데·세로형 건조기…역발상 중기 신제품 눈길 동양매직이 공중 화장...,"[세로, 건조기, 식품, 채용, 항균, 건조, 사진, 노즐, 출시, 비데]",7
...,...,...,...,...,...
1755043,NPRW1900000056.939,기술,자동차부스 찾아다닌 LG ◆ 세계 최대 가전쇼 CES 2014 ◆ LG그룹이 자동차...,"[자동차, 시장, 가전, 디스플레이, 전자, 사장, 아우디, 부스, 냉장고, 차량]",6
1540922,NPRW1900000022.1431,기술,"홀대받는 전자잉크, 아이패드 돌풍 누를까 아직도 많은 사람들이 LCD 모니터로 뭔가...","[전자, 잉크, 전자책, 단말기, 컴퓨터, 시장, 아이패드, 태블릿, 디스플레이, 패널]",6
2564984,NPRW2200000007.29616,기술,"[누리호 우주로] ‘K-발사체’ 누리호, 발사 16분간의 시나리오 ‘K-발사체’ 누...","[발사, 누리, 분리, 위성, 발사체, 추적, 궤도, 시작, 로켓, 우주]",6
3017697,NWRW1900000032.10745,기술,"SKT, 신형 스마트폰으로 KT에 대반격 ㆍ삼성전자·소니 등과 손잡고 10종 출시 ...","[아이폰, 스마트폰, 텔레콤, 전자, 신형, 구글, 애플, 삼성, 수성, 갤럭시]",6


In [32]:
df_t = df[df['match_count'] >= 1]

In [33]:
len(df_t)

237600

In [34]:
len(df)

382513

In [35]:
df_t.to_csv(directory_path+'/news_T_keyword.tsv', sep='\t', index=False)